<a href="https://colab.research.google.com/github/vishvas-chauhan/First_Repository/blob/master/Spark_SQL_hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INSTALL java and pyspark on colab

In [48]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 85 not upgraded.


In [49]:
#!sudo add-apt-repository --remove ppa:vikoadi/ppa
#!sudo apt update
#https://www-us.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz

In [50]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1//spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [51]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark=SparkSession.builder.appName('sol').getOrCreate()

## lets get started 


In [52]:
import sys, tempfile, urllib
import os

In [53]:
base_dir= '/tmp'
output_file = os.path.join(base_dir,'credit_data.csv')

In [54]:
credit_data= urllib.request.urlretrieve('http://winterolympicsmedals.com/medals.csv',output_file)

In [55]:
!ls /tmp

blockmgr-cae6601a-1bfb-4fab-91c7-fefea6d04029
credit_data.csv
dap_multiplexer.2b8d00ef72a6.root.log.INFO.20210505-102351.48
dap_multiplexer.INFO
debugger_beo3p4nfr
hsperfdata_root
initgoogle_syslog_dir.0
liblz4-java-3505556691249771352.so
liblz4-java-3505556691249771352.so.lck
spark-578b0d8b-cf54-4176-af4f-38fc13f4736e
spark-c1a0f59c-be10-43a7-ac0b-359af347c956


In [56]:
df1=spark.read.option("inferSchema","true").csv("/tmp/credit_data.csv",header=True)

In [58]:
df1.show(3)

+----+--------+-------+--------------+---+----------+------------+------+
|Year|    City|  Sport|    Discipline|NOC|     Event|Event gender| Medal|
+----+--------+-------+--------------+---+----------+------------+------+
|1924|Chamonix|Skating|Figure skating|AUT|individual|           M|Silver|
|1924|Chamonix|Skating|Figure skating|AUT|individual|           W|  Gold|
|1924|Chamonix|Skating|Figure skating|AUT|     pairs|           X|  Gold|
+----+--------+-------+--------------+---+----------+------------+------+
only showing top 3 rows



In [161]:
df=df1

In [68]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Event: string (nullable = true)
 |-- Event gender: string (nullable = true)
 |-- Medal: string (nullable = true)



In [66]:
df.select(df['City'],df['Sport']).show(2)

+--------+-------+
|    City|  Sport|
+--------+-------+
|Chamonix|Skating|
|Chamonix|Skating|
+--------+-------+
only showing top 2 rows



In [76]:
df.filter(df['Medal']=='Gold').show(3)

+----+--------+----------+--------------+---+----------+------------+-----+
|Year|    City|     Sport|    Discipline|NOC|     Event|Event gender|Medal|
+----+--------+----------+--------------+---+----------+------------+-----+
|1924|Chamonix|   Skating|Figure skating|AUT|individual|           W| Gold|
|1924|Chamonix|   Skating|Figure skating|AUT|     pairs|           X| Gold|
|1924|Chamonix|Ice Hockey|    Ice Hockey|CAN|ice hockey|           M| Gold|
+----+--------+----------+--------------+---+----------+------------+-----+
only showing top 3 rows



In [77]:
df.groupby('Medal').count().show()

+------+-----+
| Medal|count|
+------+-----+
|Silver|  773|
|  Gold|  774|
|Bronze|  764|
+------+-----+



In [ ]:
from google.colab import files
f=files.upload()

In [23]:
dataset = spark.read.csv('Sales by SKU (29).csv',inferSchema=True, header =True)

In [78]:
df1.show(5)

+----+--------+----------+--------------+---+----------+------------+------+
|Year|    City|     Sport|    Discipline|NOC|     Event|Event gender| Medal|
+----+--------+----------+--------------+---+----------+------------+------+
|1924|Chamonix|   Skating|Figure skating|AUT|individual|           M|Silver|
|1924|Chamonix|   Skating|Figure skating|AUT|individual|           W|  Gold|
|1924|Chamonix|   Skating|Figure skating|AUT|     pairs|           X|  Gold|
|1924|Chamonix| Bobsleigh|     Bobsleigh|BEL|  four-man|           M|Bronze|
|1924|Chamonix|Ice Hockey|    Ice Hockey|CAN|ice hockey|           M|  Gold|
+----+--------+----------+--------------+---+----------+------------+------+
only showing top 5 rows



In [79]:
# Register the DataFrame as a SQL temporary view
df1.createOrReplaceTempView("medals")



In [80]:
sqlview= spark.sql("select * from medals")
sqlview.show(4)

+----+--------+---------+--------------+---+----------+------------+------+
|Year|    City|    Sport|    Discipline|NOC|     Event|Event gender| Medal|
+----+--------+---------+--------------+---+----------+------------+------+
|1924|Chamonix|  Skating|Figure skating|AUT|individual|           M|Silver|
|1924|Chamonix|  Skating|Figure skating|AUT|individual|           W|  Gold|
|1924|Chamonix|  Skating|Figure skating|AUT|     pairs|           X|  Gold|
|1924|Chamonix|Bobsleigh|     Bobsleigh|BEL|  four-man|           M|Bronze|
+----+--------+---------+--------------+---+----------+------------+------+
only showing top 4 rows



In [91]:
spark.sql("select City,Medal from medals where Sport=='Skating' and Medal=='Silver'").show()

+-----------+------+
|       City| Medal|
+-----------+------+
|   Chamonix|Silver|
|   Chamonix|Silver|
|   Chamonix|Silver|
|   Chamonix|Silver|
|   Chamonix|Silver|
|   Chamonix|Silver|
|   Chamonix|Silver|
|   Chamonix|Silver|
| St. Moritz|Silver|
| St. Moritz|Silver|
| St. Moritz|Silver|
| St. Moritz|Silver|
| St. Moritz|Silver|
|Lake Placid|Silver|
|Lake Placid|Silver|
|Lake Placid|Silver|
|Lake Placid|Silver|
|Lake Placid|Silver|
|Lake Placid|Silver|
|Lake Placid|Silver|
+-----------+------+
only showing top 20 rows



In [94]:
spark.sql("select City as c,Medal as m from medals where Sport=='Skating'").show()

+--------+------+
|       c|     m|
+--------+------+
|Chamonix|Silver|
|Chamonix|  Gold|
|Chamonix|  Gold|
|Chamonix|Silver|
|Chamonix|  Gold|
|Chamonix|Silver|
|Chamonix|  Gold|
|Chamonix|  Gold|
|Chamonix|Silver|
|Chamonix|Bronze|
|Chamonix|Bronze|
|Chamonix|  Gold|
|Chamonix|Bronze|
|Chamonix|Bronze|
|Chamonix|Bronze|
|Chamonix|Bronze|
|Chamonix|Silver|
|Chamonix|Bronze|
|Chamonix|Bronze|
|Chamonix|Silver|
+--------+------+
only showing top 20 rows



In [115]:
spark.sql("select count(City),Medal from medals where Sport=='Skating' group by Medal order by Medal").show()

+-----------+------+
|count(City)| Medal|
+-----------+------+
|        249|Bronze|
|        254|  Gold|
|        255|Silver|
+-----------+------+



In [143]:
#find sports played more than 100 times
spark.sql("select Sport,total from (select Sport,count(Sport) as total from medals group by Sport) where total > 100 order by total").show()

+---------+-----+
|    Sport|total|
+---------+-----+
|     Luge|  108|
|Bobsleigh|  133|
| Biathlon|  162|
|  Skating|  758|
|   Skiing| 1060|
+---------+-----+



In [144]:
spark.sql("select Sport, count(Sport) as total from medals group by Sport having total >100").show()

+---------+-----+
|    Sport|total|
+---------+-----+
| Biathlon|  162|
|  Skating|  758|
|   Skiing| 1060|
|Bobsleigh|  133|
|     Luge|  108|
+---------+-----+



In [163]:
df.show(4)

+----+--------+---------+--------------+---+----------+------------+------+
|Year|    City|    Sport|    Discipline|NOC|     Event|Event gender| Medal|
+----+--------+---------+--------------+---+----------+------------+------+
|1924|Chamonix|  Skating|Figure skating|AUT|individual|           M|Silver|
|1924|Chamonix|  Skating|Figure skating|AUT|individual|           W|  Gold|
|1924|Chamonix|  Skating|Figure skating|AUT|     pairs|           X|  Gold|
|1924|Chamonix|Bobsleigh|     Bobsleigh|BEL|  four-man|           M|Bronze|
+----+--------+---------+--------------+---+----------+------------+------+
only showing top 4 rows



In [167]:
from pyspark.sql.functions import *
df_addcol= df.withColumn('Mango',lit(0))
df.withColumn("gold", when(df.Medal=='Gold',"hero").otherwise("alright")).show()

+----+--------+----------+---------------+---+-------------------+------------+------+-------+
|Year|    City|     Sport|     Discipline|NOC|              Event|Event gender| Medal|   gold|
+----+--------+----------+---------------+---+-------------------+------------+------+-------+
|1924|Chamonix|   Skating| Figure skating|AUT|         individual|           M|Silver|alright|
|1924|Chamonix|   Skating| Figure skating|AUT|         individual|           W|  Gold|   hero|
|1924|Chamonix|   Skating| Figure skating|AUT|              pairs|           X|  Gold|   hero|
|1924|Chamonix| Bobsleigh|      Bobsleigh|BEL|           four-man|           M|Bronze|alright|
|1924|Chamonix|Ice Hockey|     Ice Hockey|CAN|         ice hockey|           M|  Gold|   hero|
|1924|Chamonix|  Biathlon|       Biathlon|FIN|    military patrol|           M|Silver|alright|
|1924|Chamonix|   Skating| Figure skating|FIN|              pairs|           X|Silver|alright|
|1924|Chamonix|   Skating|  Speed skating|FIN|    

In [166]:
df_addcol.show()

+----+--------+----------+---------------+---+-------------------+------------+------+-----+
|Year|    City|     Sport|     Discipline|NOC|              Event|Event gender| Medal|Mango|
+----+--------+----------+---------------+---+-------------------+------------+------+-----+
|1924|Chamonix|   Skating| Figure skating|AUT|         individual|           M|Silver|    0|
|1924|Chamonix|   Skating| Figure skating|AUT|         individual|           W|  Gold|    0|
|1924|Chamonix|   Skating| Figure skating|AUT|              pairs|           X|  Gold|    0|
|1924|Chamonix| Bobsleigh|      Bobsleigh|BEL|           four-man|           M|Bronze|    0|
|1924|Chamonix|Ice Hockey|     Ice Hockey|CAN|         ice hockey|           M|  Gold|    0|
|1924|Chamonix|  Biathlon|       Biathlon|FIN|    military patrol|           M|Silver|    0|
|1924|Chamonix|   Skating| Figure skating|FIN|              pairs|           X|Silver|    0|
|1924|Chamonix|   Skating|  Speed skating|FIN|             10000m|    

In [ ]:
spark.sql("ALTER TABLE medals ADD EmployeeID numeric NOT NULL IDENTITY (1, 1) ALTER TABLE Employees ADD CONSTRAINT PK_Employees PRIMARY KEY CLUSTERED (EmployeeID) WITH( STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]").show()

## time to create dataframe 

In [146]:
#Sample Dataframe
from pyspark.sql.types import *
df=spark.createDataFrame([(1,"[{'code':'1234','name':'manoj'},{'code':'124','name':'kumar'},{'code':'4567','name':'dhakad'}]",),(2,"[{'code':'97248','name':'joe'},{'code':'2424','name':'alice'},{'code':'464','name':'bob'}]",)],["id","classification",])


In [148]:
df.show()

+---+--------------------+
| id|      classification|
+---+--------------------+
|  1|[{'code':'1234','...|
|  2|[{'code':'97248',...|
+---+--------------------+



In [33]:
words = "to be or not to be".split()
words

['to', 'be', 'or', 'not', 'to', 'be']